In [101]:
### user features ###
############# user related feature   #############
"""
3.user related: 
      count_merchant. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      buy_use_coupon/buy_total
      user_date_datereceived_gap
      
"""
import pandas as pd
import numpy as np
from datetime import date
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

In [102]:
off_train = spark.read.csv('/Users/sogmac/Downloads/ccf_offline_stage1_train-2.csv',header=True, inferSchema=True)
off_train.header = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_test = spark.read.csv('/Users/sogmac/Downloads/ccf_offline_stage1_test_revised.csv', header=True, inferSchema=True)
off_test.header = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
on_train = spark.read.csv('/Users/sogmac/Downloads/ccf_online_stage1_train.csv', header= True, inferSchema=True)
on_train.header = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

In [103]:
'''
dataset split:
                      (date_received)                              
           dateset3: 20160701~20160731 (113640),features3 from 20160315~20160630  (off_test)
           dateset2: 20160515~20160615 (25846),features2 from 20160201~20160514  
           dateset1: 20160414~20160514 (138303),features1 from 20160101~20160413 
'''
dataset3 = off_test
feature3 = off_train.filter(((off_train.Date>='20160315')&(off_train.Date<='20160630'))\
                           |((off_train.Date=='null') & (off_train.Date_received>='20160315')&(off_train.Date_received<='20160630')))\
                        .select('*')
dataset2 = off_train.filter((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615')).select('*')
feature2 = off_train.filter((off_train.Date>='20160201')&(off_train.Date<='20160514')|((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514'))).select('*')
dataset1 = off_train.filter((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514')).select('*')
feature1 = off_train.filter((off_train.Date>='20160101')&(off_train.Date<='20160413')|((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413'))).select('*')

In [104]:
print(feature3.count())
print(dataset2.count())
print(feature2.count())
print(dataset1.count())
print(feature1.count())

1036975
258446
812779
137167
995240


In [106]:
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    return (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days

In [107]:
user3 = feature3[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user3[['user_id']]
t0 = t.distinct()

t1 = user3[user3.date!='null'][['user_id','merchant_id']] 
t1 = t1.distinct()
t1 = t1.withColumn('merchant_id',lit(1))
t1 = t1.groupBy('user_id').sum('merchant_id').withColumnRenamed('sum(merchant_id)', 'count_merchant')
#print(t1.count()) 180561
#t1.orderBy('user_id').show()


In [108]:
t2 = user3[(user3.date!='null')&(user3.coupon_id!='null')][['user_id','distance']]
t2 = t2.replace('null','-1')
t2 = t2.select('user_id',t2['distance'].cast('int'))
t2 = t2.replace(-1,np.nan)
t3 = t2.groupBy('user_id').min('distance').withColumnRenamed('min(distance)', 'user_min_distance')
t4 = t2.groupby('user_id').max('distance').withColumnRenamed('max(distance)', 'user_max_distance')
t5 = t2.groupby('user_id').avg('distance').withColumnRenamed('avg(distance)', 'user_mean_distance')
med = expr('percentile_approx(Distance,0.5)')
t6 = t2.groupby('user_id').agg(med.alias('user_median_distance'))

In [112]:
t7 = user3[(user3.date!='null')&(user3.coupon_id!='null')][['user_id']]
t7 = t7.withColumn('buy_use_coupon1',lit(1))
t7 = t7.groupBy('user_id').sum('buy_use_coupon1').withColumnRenamed('sum(buy_use_coupon1)', 'buy_use_coupon')

t8 = user3[user3.date!='null'][['user_id']]
t8 = t8.withColumn('buy_total1',lit(1))
t8 = t8.groupBy('user_id').sum('buy_total1').withColumnRenamed('sum(buy_total1)', 'buy_total')

t9 = user3[user3.coupon_id!='null'][['user_id']]
t9 = t9.withColumn('coupon_received1',lit(1))
t9 = t9.groupBy('user_id').sum('coupon_received1').withColumnRenamed('sum(coupon_received1)', 'coupon_received')

In [113]:
t10 = user3[(user3.date_received!='null')&(user3.date!='null')][['user_id','date_received','date']]
t10 = t10.withColumn("date",lit(to_date(t10.date,"yyyyMMdd")))
t10 = t10.withColumn("date_received",lit(to_date(t10.date_received,"yyyyMMdd")))
t10 = t10.withColumn('user_date_datereceived_gap', lit(datediff('date','date_received')))

t11 = t10.groupby('user_id').avg('user_date_datereceived_gap').withColumnRenamed('avg(user_date_datereceived_gap)', 'avg_user_date_datereceived_gap')
t12 = t10.groupby('user_id').min('user_date_datereceived_gap').withColumnRenamed('min(user_date_datereceived_gap)', 'min_user_date_datereceived_gap')
t13 = t10.groupby('user_id').max('user_date_datereceived_gap').withColumnRenamed('max(user_date_datereceived_gap)', 'max_user_date_datereceived_gap')


In [114]:
user3_feature = t.join(t1,'user_id','left')
user3_feature = user3_feature.join(t3,'user_id','left')
user3_feature = user3_feature.join(t4,'user_id','left')
user3_feature = user3_feature.join(t5,'user_id','left')
user3_feature = user3_feature.join(t6,'user_id','left')
user3_feature = user3_feature.join(t7,'user_id','left')
user3_feature = user3_feature.join(t8,'user_id','left')
user3_feature = user3_feature.join(t9,'user_id','left')
user3_feature = user3_feature.join(t11,'user_id','left')
user3_feature = user3_feature.join(t12,'user_id','left')
user3_feature = user3_feature.join(t13,'user_id','left')

In [115]:
user3_feature = user3_feature.na.fill(value = 0, subset =['count_merchant','buy_use_coupon','buy_total','coupon_received'])
user3_feature = user3_feature.withColumn('buy_use_coupon_rate',\
                                         lit((user3_feature.buy_use_coupon.cast('float'))/user3_feature.buy_total.cast('float')))

In [116]:
user3_feature.coalesce(1).write.mode("overwrite")\
.option('mapreduce.fileoutputcommitter.marksuccessfuljobs','false')\
.option('header','true').csv('/Users/sogmac/Desktop/user3_feature')

In [122]:
user2 = feature2[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user2[['user_id']]
t0 = t.distinct()

t1 = user2[user2.date!='null'][['user_id','merchant_id']] 
t1 = t1.distinct()
t1 = t1.withColumn('merchant_id',lit(1))
t1 = t1.groupBy('user_id').sum('merchant_id').withColumnRenamed('sum(merchant_id)', 'count_merchant')

t2 = user2[(user2.date!='null')&(user2.coupon_id!='null')][['user_id','distance']]
t2 = t2.replace('null','-1')
t2 = t2.select('user_id',t2['distance'].cast('int'))
t2 = t2.replace(-1,np.nan)
t3 = t2.groupBy('user_id').min('distance').withColumnRenamed('min(distance)', 'user_min_distance')
t4 = t2.groupby('user_id').max('distance').withColumnRenamed('max(distance)', 'user_max_distance')
t5 = t2.groupby('user_id').avg('distance').withColumnRenamed('avg(distance)', 'user_mean_distance')

med = expr('percentile_approx(Distance,0.5)')
t6 = t2.groupby('user_id').agg(med.alias('user_median_distance'))

t7 = user2[(user2.date!='null')&(user2.coupon_id!='null')][['user_id']]
t7 = t7.withColumn('buy_use_coupon1',lit(1))
t7 = t7.groupBy('user_id').sum('buy_use_coupon1').withColumnRenamed('sum(buy_use_coupon1)', 'buy_use_coupon')

t8 = user2[user2.date!='null'][['user_id']]
t8 = t8.withColumn('buy_total1',lit(1))
t8 = t8.groupBy('user_id').sum('buy_total1').withColumnRenamed('sum(buy_total1)', 'buy_total')

t9 = user2[user2.coupon_id!='null'][['user_id']]
t9 = t9.withColumn('coupon_received1',lit(1))
t9 = t9.groupBy('user_id').sum('coupon_received1').withColumnRenamed('sum(coupon_received1)', 'coupon_received')

t10 = user2[(user2.date_received!='null')&(user2.date!='null')][['user_id','date_received','date']]
t10 = t10.withColumn("date",lit(to_date(t10.date,"yyyyMMdd")))
t10 = t10.withColumn("date_received",lit(to_date(t10.date_received,"yyyyMMdd")))
t10 = t10.withColumn('user_date_datereceived_gap', lit(datediff('date','date_received')))

t11 = t10.groupby('user_id').avg('user_date_datereceived_gap').withColumnRenamed('avg(user_date_datereceived_gap)', 'avg_user_date_datereceived_gap')
t12 = t10.groupby('user_id').min('user_date_datereceived_gap').withColumnRenamed('min(user_date_datereceived_gap)', 'min_user_date_datereceived_gap')
t13 = t10.groupby('user_id').max('user_date_datereceived_gap').withColumnRenamed('max(user_date_datereceived_gap)', 'max_user_date_datereceived_gap')

user2_feature = t.join(t1,'user_id','left')
user2_feature = user2_feature.join(t3,'user_id','left')
user2_feature = user2_feature.join(t4,'user_id','left')
user2_feature = user2_feature.join(t5,'user_id','left')
user2_feature = user2_feature.join(t6,'user_id','left')
user2_feature = user2_feature.join(t7,'user_id','left')
user2_feature = user2_feature.join(t8,'user_id','left')
user2_feature = user2_feature.join(t9,'user_id','left')
user2_feature = user2_feature.join(t11,'user_id','left')
user2_feature = user2_feature.join(t12,'user_id','left')
user2_feature = user2_feature.join(t13,'user_id','left')

In [123]:
user2_feature = user2_feature.na.fill(value = 0, subset =['count_merchant','buy_use_coupon','buy_total','coupon_received'])
user2_feature = user2_feature.withColumn('buy_use_coupon_rate',\
                                         lit((user2_feature.buy_use_coupon.cast('float'))/user2_feature.buy_total.cast('float')))
user2_feature = user2_feature.withColumn('user_coupon_transfer_rate',\
                                         lit((user2_feature.buy_use_coupon.cast('float'))/user2_feature.coupon_received.cast('float')))


In [124]:
user2_feature.coalesce(1).write.mode("overwrite")\
.option('mapreduce.fileoutputcommitter.marksuccessfuljobs','false')\
.option('header','true').csv('/Users/sogmac/Desktop/user2_feature')

In [125]:
user1 = feature1[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user1[['user_id']]
t0 = t.distinct()

t1 = user1[user1.date!='null'][['user_id','merchant_id']] 
t1 = t1.distinct()
t1 = t1.withColumn('merchant_id',lit(1))
t1 = t1.groupBy('user_id').sum('merchant_id').withColumnRenamed('sum(merchant_id)', 'count_merchant')

t2 = user1[(user1.date!='null')&(user1.coupon_id!='null')][['user_id','distance']]
t2 = t2.replace('null','-1')
t2 = t2.select('user_id',t2['distance'].cast('int'))
t2 = t2.replace(-1,np.nan)
t3 = t2.groupBy('user_id').min('distance').withColumnRenamed('min(distance)', 'user_min_distance')
t4 = t2.groupby('user_id').max('distance').withColumnRenamed('max(distance)', 'user_max_distance')
t5 = t2.groupby('user_id').avg('distance').withColumnRenamed('avg(distance)', 'user_mean_distance')

med = expr('percentile_approx(Distance,0.5)')
t6 = t2.groupby('user_id').agg(med.alias('user_median_distance'))

t7 = user1[(user1.date!='null')&(user1.coupon_id!='null')][['user_id']]
t7 = t7.withColumn('buy_use_coupon1',lit(1))
t7 = t7.groupBy('user_id').sum('buy_use_coupon1').withColumnRenamed('sum(buy_use_coupon1)', 'buy_use_coupon')

t8 = user1[user1.date!='null'][['user_id']]
t8 = t8.withColumn('buy_total1',lit(1))
t8 = t8.groupBy('user_id').sum('buy_total1').withColumnRenamed('sum(buy_total1)', 'buy_total')

t9 = user1[user1.coupon_id!='null'][['user_id']]
t9 = t9.withColumn('coupon_received1',lit(1))
t9 = t9.groupBy('user_id').sum('coupon_received1').withColumnRenamed('sum(coupon_received1)', 'coupon_received')

t10 = user1[(user1.date_received!='null')&(user1.date!='null')][['user_id','date_received','date']]
t10 = t10.withColumn("date",lit(to_date(t10.date,"yyyyMMdd")))
t10 = t10.withColumn("date_received",lit(to_date(t10.date_received,"yyyyMMdd")))
t10 = t10.withColumn('user_date_datereceived_gap', lit(datediff('date','date_received')))

t11 = t10.groupby('user_id').avg('user_date_datereceived_gap').withColumnRenamed('avg(user_date_datereceived_gap)', 'avg_user_date_datereceived_gap')
t12 = t10.groupby('user_id').min('user_date_datereceived_gap').withColumnRenamed('min(user_date_datereceived_gap)', 'min_user_date_datereceived_gap')
t13 = t10.groupby('user_id').max('user_date_datereceived_gap').withColumnRenamed('max(user_date_datereceived_gap)', 'max_user_date_datereceived_gap')

user1_feature = t.join(t1,'user_id','left')
user1_feature = user1_feature.join(t3,'user_id','left')
user1_feature = user1_feature.join(t4,'user_id','left')
user1_feature = user1_feature.join(t5,'user_id','left')
user1_feature = user1_feature.join(t6,'user_id','left')
user1_feature = user1_feature.join(t7,'user_id','left')
user1_feature = user1_feature.join(t8,'user_id','left')
user1_feature = user1_feature.join(t9,'user_id','left')
user1_feature = user1_feature.join(t11,'user_id','left')
user1_feature = user1_feature.join(t12,'user_id','left')
user1_feature = user1_feature.join(t13,'user_id','left')

In [126]:
user1_feature = user1_feature.na.fill(value = 0, subset =['count_merchant','buy_use_coupon','buy_total','coupon_received'])
user1_feature = user1_feature.withColumn('buy_use_coupon_rate',\
                                         lit((user1_feature.buy_use_coupon.cast('float'))/user1_feature.buy_total.cast('float')))
user1_feature = user1_feature.withColumn('user_coupon_transfer_rate',\
                                         lit((user1_feature.buy_use_coupon.cast('float'))/user1_feature.coupon_received.cast('float')))

In [127]:
user1_feature.coalesce(1).write.mode("overwrite")\
.option('mapreduce.fileoutputcommitter.marksuccessfuljobs','false')\
.option('header','true').csv('/Users/sogmac/Desktop/user1_feature')

import matplotlib.pyplot as plt
from matplotlib.font_manager import _rebuild
_rebuild()
# 正常显示中文
from pylab import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
# 正常显示符号
matplotlib.rcParams['axes.unicode_minus']=False
import seaborn as sns
import numpy as np
from pyspark.sql.functions import *
#print(matplotlib.matplotlib_fname())

ontr = spark.read.csv('/Users/sogmac/Downloads/ccf_online_stage1_train.csv', header=True, inferSchema=True)
offtr = spark.read.csv('/Users/sogmac/Downloads/ccf_offline_stage1_train-2.csv', header=True, inferSchema=True)
offtest = spark.read.csv('/Users/sogmac/Downloads/ccf_offline_stage1_test_revised.csv', header=True, inferSchema=True)
#print(ontr.distinct().count())
#print(offtr.count())
#offtest.show()

def float2txt(x):
    try:
        yield str(int(x))
    except:
        yield x

def discount2txt(x):
    try:
        yield int(x)
    except:
        yield x

#总用户数
#alluser = print(ontr.distinct().count())
#总消费用户数
#allcos = print(offtr.distinct().count())
#有优惠券，15天内消费
u1 = offtr.filter("Coupon_id != 'null'").filter("Date != 'null'").filter("Date - Date_received <=15")
#有优惠券，15天外消费
u1_1 = offtr.filter("Coupon_id != 'null'").filter("Date != 'null'").filter("Date - Date_received >15")
pos = offtr.filter("Coupon_id != 'null'").filter("Date != 'null'")
#有优惠券，不消费
u2 = offtr.filter("Coupon_id != 'null'").filter("Date == 'null'")
#没优惠券，但消费
u3 = offtr.filter("Coupon_id == 'null'").filter("Date != 'null'")
#没优惠券，不消费
u4 = offtr.filter("Coupon_id == 'null'").filter("Date == 'null'")
print("有优惠券，15天内消费:",u1.count(),'\n',
      "有优惠券，15天外消费:",u1_1.count(),'\n',
      "有优惠券，不消费:",u2.count(),'\n',
      "没优惠券，但消费:",u3.count(),'\n',
      "没优惠券，不消费:",u4.count())

#有优惠券，15天内消费: 51375 
#有优惠券，15天外消费: 24007 
#有优惠券，不消费: 977900 
#没优惠券，但消费: 701602 
#没优惠券，不消费: 0

u = [u1.count(),u1_1.count(),u2.count(),u3.count(),u4.count()]
ulabel = ["有优惠券，15天内消费","有优惠券，15天外消费","有优惠券，不用券消费","没有优惠券，直接消费","没有优惠券，不消费"]
plt.pie(u,labels=ulabel,autopct='%.0f%%', textprops = {'fontsize':12, 'color':'k'},\
        wedgeprops={'alpha':0.8},colors=sns.color_palette('summer_r',n_colors=5))
#plt.savefig('people_distribution.jpg')
plt.show()

sample_count = [u[0]+u[1],u[3],u[2]]
sample_label = ['positive','0','negtive']
plt.pie(sample_count,labels = sample_label,autopct='%.0f%%', textprops = {'fontsize':12, 'color':'k'},\
        wedgeprops={'alpha':0.8},colors=sns.color_palette('summer_r',n_colors=3))
#plt.savefig('sample_distribution.jpg')
plt.show()

#u2.groupBy('Distance').count().show()
#u2.groupBy('Discount_rate').count().show()